## VARCHAR(M)

In [1]:
def calcVarcharColSize(M, S, charcterset):
    
    charctersetList = ['latin1', 'usc2', 'utf8', 'utf8mb3', 'utf8mb4']
    if not charcterset in charctersetList:
        return None
        
    alpha = -1
    if charcterset == 'latin1':
        alpha = 1
    elif charcterset == 'usc2':
        alpha = 2
    elif charcterset == 'utf8' or  charcterset == 'utf8mb3':
        alpha = 3
    elif charcterset == 'utf8mb4':
        alpha = 4

    L_m = M * alpha
    L_s = S * alpha
    
    return  L_s + 1 if L_m <= 255 else L_s + 2 

In [2]:
print(calcVarcharColSize(255, 4, 'latin1'))
print(calcVarcharColSize(255, 4, 'usc2'))
print(calcVarcharColSize(255, 4, 'usc3'))

5
10
None


## INT

In [3]:
def calcIntColSize():
    return 4

## DATE, DATETIME

In [4]:
def calcDateColSize(fractionalSecondsPrecision):
    fractionalSecondStorage = 0
    if(fractionalSecondsPrecision in range(1,3)):
        fractionalSecondStorage = 1
    elif(fractionalSecondsPrecision in range(3,5)):
        fractionalSecondStorage = 2
    elif(fractionalSecondsPrecision in range(5,7)):
        fractionalSecondStorage = 3
    return 3 + fractionalSecondStorage

In [5]:
print(calcDateColSize(0))
print(calcDateColSize(2))
print(calcDateColSize(4))
print(calcDateColSize(6))

3
4
5
6


In [6]:
def calcDatetimeColSize(fractionalSecondsPrecision):
    fractionalSecondStorage = 0
    if(fractionalSecondsPrecision in range(1,3)):
        fractionalSecondStorage = 1
    elif(fractionalSecondsPrecision in range(3,5)):
        fractionalSecondStorage = 2
    elif(fractionalSecondsPrecision in range(5,7)):
        fractionalSecondStorage = 3
    return 5 + fractionalSecondStorage

## Estimate column size in bytes

calculate total column size of schema

In [7]:
import const

const.VARCHAR = 'varchar'
const.INT = 'int'
const.DATE = 'date'
const.DATETIME = 'datetime'

In [8]:
def calcTotalColSize(types, characterset):
    totalColSize = 0
    for type in types:
        if type[0] == const.VARCHAR:
            totalColSize += calcVarcharColSize(type[1], type[1], characterset)
        elif type[0] == const.INT:
            totalColSize += calcIntColSize()
        elif type[0] == const.DATE:
            totalColSize += calcDateColSize(type[1])
        elif type[0] == const.DATETIME:
            totalColSize += calcDatetimeColSize(type[1])
    return totalColSize

In [9]:
%%html
<style>
table {float:left}
</style>

https://stackoverflow.com/questions/21892570/ipython-notebook-align-table-to-the-left-of-cell

## How to use

Estimate data size of following schema(utf8)

schema1

|  col1         | col2   | col3  | col4     |
| :--------------| :------ |:-------|:----------|
|  VARCHAR(13)  |  INT   | DATE  | DATETIME |

### 1. Estimate total column size in bytes
- VARCHAR(M)-> ('varchar', M)
- INT -> ('int',)
- DATE(n) -> ('date', n)
- DATETIME(n) -> ('datetime', n)

In [10]:
types = [(const.VARCHAR, 13), (const.INT,), (const.DATE, 0), (const.DATETIME, 0)]
totalColSize = calcTotalColSize(types, 'utf8')

print("Total column size: {} {}".format(totalColSize, "bytes"))

Total column size: 52 bytes


### 2. Estimate number of rows

In [11]:
estimatedRows = 100
print("Estimated number of rows: {}".format(estimatedRows))

Estimated number of rows: 100


### 3. Index factor

In [12]:
indexFactor = 2
print("Index factor: {}".format(indexFactor))

Index factor: 2


### 4. Overhead factor

In [13]:
overheadFactor = 1.2
print("Overhead factor: {}".format(overheadFactor))

Overhead factor: 1.2


### 5. total column

In [16]:
import pandas as pd

# | schema name | estimatied total column size | estimatied rows | index factor | overhead factor | schema data size |

schemaNameVal = ['table1']
estimatedTotalColSizeVal = [totalColSize]
estiamtedRowsVal = [estimatedRows]
indexFactorVal = [indexFactor]
overheadFactorVal = [overheadFactor]
schemaDataSizeVal = [totalColSize * estimatedRows * indexFactor * overheadFactor]

data_dict = {
    'schema name':schemaNameVal,
    'estimatied total column size': estimatedTotalColSizeVal,
    'estimatied rows': estiamtedRowsVal,
    'index factor': indexFactorVal,
    'overhead factor': overheadFactorVal,
    'estimatied schema data size': schemaDataSizeVal
}

df = pd.DataFrame.from_dict(data_dict)
df

,schema name,estimatied total column size,estimatied rows,index factor,overhead factor,estimatied schema data size
0,table1,52,100,2,1.2,12480.0
